# Extracción de datos

En esta sección se extraerá los datos de la URL citada a continuación. Los indicadores tomados en cuenta son: 
* Edad de la población
* Renta Media
* Niveles de Educación
* Extranjeros
* Tamaño medio del Hogar
* Número de Desempleados
> https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=71359583a773a510VgnVCM2000001f4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default*

In [122]:
import pandas as pd

In [123]:
# Función Genérica para iterar fuente 
# Lectura iterativa de excel
def iteracion_callback(contYear, indicadorName, contadorInicial, contadorFinal, unidad): 
    sheetNamesLst=['CENTRO', 'ARGANZUELA', 'RETIRO', 'SALAMANCA', 'CHAMARTÍN', 'TETUÁN', 'CHAMBERÍ', 'FUENCARRAL-EL PARDO','MONCLOA-ARAVACA', 'LATINA', 'CARABANCHEL', 'USERA', 'PUENTE DE VALLECAS', 'MORATALAZ', 'CIUDAD LINEAL', 'HORTALEZA', 'VILLAVERDE', 'VILLA DE VALLECAS', 'VICÁLVARO','SAN BLAS-CANILLEJAS','BARAJAS']
    arrayFullEdad = []
    dfPreProcesadoFull = pd.DataFrame()
    # Itera Paginas del excel
    for x, nameSheet in enumerate(sheetNamesLst):   
        dfAux = pd.DataFrame()
        df2 = pd.DataFrame()        
        #Lectura de excel
        fileName = ''
        fileName = '../../original_data/panel_indicadores_distritos_barrios_' + str(contYear) + '.xls'
        df = pd.read_excel(fileName, 
                           sheet_name=nameSheet, 
                           skiprows= 1)
        # Se posiciona en filas de edad en caso de la 
        # última página seteamos el número de columnas a leer por existir 
        # más columnas basura
        if x == 20 :
            dfAux=df.iloc[contadorInicial:contadorInicial+contadorFinal, 0:16]            
        else:
            dfAux=df.iloc[contadorInicial:contadorInicial+contadorFinal]            

        dfAux.reset_index(inplace=True, drop=True)
        
        columnasNombre = []
        contadorFor=0
        # Itera columnas para controlar celdas combinadas
        for col in dfAux.columns:
            if contadorFor == 0 :
                newCol = indicadorName
            elif contadorFor % 2 == 0:
                newCol = nomColAux.upper()
            else :
                newCol = 'GT_' + col
            columnasNombre.append(newCol)
            nomColAux = col
            contadorFor = contadorFor + 1
        # Se coloca los nuevos nombres de columnas
        dfAux.columns = columnasNombre
        
        indexColumnasBorrar = []
        contadorColumnas = 0
        # Obtener indices de las columnas para ser borradas las de porcentaje
        for row in columnasNombre:
            if contadorColumnas % 2 != 0 :
                indexColumnasBorrar.append(contadorColumnas)
            contadorColumnas = contadorColumnas + 1
        dfAux.drop(dfAux.columns[indexColumnasBorrar], axis = 1, inplace=True)
        dfAux.drop(dfAux.columns[[1]], axis = 1, inplace=True)
        
        # Pasar columnas como valores del dataframe bajo el campo Barrios
        df2 = pd.melt(dfAux, id_vars=[indicadorName], 
                      var_name="BARRIOS", value_name=unidad)
        # Strings a mayúsculas
        df2.columns = df2.columns.str.upper()
        # Colocamos el anio correspodniente
        df2['ANIO'] = contYear
        
        arrayFullEdad.append(df2)
        
    dfPreProcesadoFull = pd.concat(arrayFullEdad)                
    
    return dfPreProcesadoFull

## Edad de la población periodo 2016, 2018, 2020

In [124]:
# Dataset Nivel 1
anio_indicador = 2016
dfEdad2016 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 7, 6, 'NRO_PERSONAS')
dfEdad2016.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfEdad2018 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 9, 6, 'NRO_PERSONAS')
dfEdad2018.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfEdad2020 = iteracion_callback(anio_indicador, 'RANGO_EDAD', 9, 6, 'NRO_PERSONAS')
dfEdad2020.to_excel("../../data_preprocesada_N1/edad_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfEdad2016, dfEdad2018, dfEdad2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/edad_barrios_madrid.xlsx")


/Users/renangfz/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## Renta neta media anual de la población periodo 2016, 2018, 2020

In [125]:
# Dataset Nivel 1
anio_indicador = 2016
dfRenta2016 = iteracion_callback(anio_indicador, 'INDICADOR', 40, 1, 'RENTA_MEDIA')
dfRenta2016.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfRenta2018 = iteracion_callback(anio_indicador, 'INDICADOR', 53, 1, 'RENTA_MEDIA')
dfRenta2018.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfRenta2020 = iteracion_callback(anio_indicador, 'INDICADOR', 53, 1, 'RENTA_MEDIA')
dfRenta2020.to_excel("../../data_preprocesada_N1/renta_neta_media_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfRenta2016, dfRenta2018, dfRenta2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/renta_neta_media_barrios_madrid.xlsx")


## Nivel Educación población periodo 2016, 2018, 2020

In [126]:
# Dataset Nivel 1
anio_indicador = 2016
dfNivelEducacion2016 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 98, 5, 'NRO_PERSONAS')
dfNivelEducacion2016.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfNivelEducacion2018 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 113, 6, 'NRO_PERSONAS')
dfNivelEducacion2018.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfNivelEducacion2020 = iteracion_callback(anio_indicador, 'NIVELES_EDUCACION', 113, 6, 'NRO_PERSONAS')
dfNivelEducacion2020.to_excel("../../data_preprocesada_N1/nivel_educación_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfNivelEducacion2016, dfNivelEducacion2018, dfNivelEducacion2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/nivel_educación_barrios_madrid.xlsx")


## Número de Extranjeros 2016, 2018, 2020

In [127]:
# Dataset Nivel 1
anio_indicador = 2016
dfNroExtranjeros2016 = iteracion_callback(anio_indicador, 'INDICADOR', 20, 1, 'NRO_PERSONAS')
dfNroExtranjeros2016.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfNroExtranjeros2018 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_PERSONAS')
dfNroExtranjeros2018.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfNroExtranjeros2020 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_PERSONAS')
dfNroExtranjeros2020.to_excel("../../data_preprocesada_N1/nro_extranjeros_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfNroExtranjeros2016, dfNroExtranjeros2018, dfNroExtranjeros2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/nro_extranjeros_barrios_madrid.xlsx")

## Tamaño medio del hogar 2016, 2018, 2020

In [128]:
# Dataset Nivel 1
anio_indicador = 2016
dfSizeHogar2016 = iteracion_callback(anio_indicador, 'INDICADOR', 27, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2016.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfSizeHogar2018 = iteracion_callback(anio_indicador, 'INDICADOR', 35, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2018.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfSizeHogar2020 = iteracion_callback(anio_indicador, 'INDICADOR', 35, 1, 'NRO_MEDIA_PERSONAS')
dfSizeHogar2020.to_excel("../../data_preprocesada_N1/size_medio_hogares_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfSizeHogar2016, dfSizeHogar2018, dfSizeHogar2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/size_medio_hogares_barrios_madrid.xlsx")

## Número de desempleados 2016, 2018, 2020

In [129]:
# Dataset Nivel 1
anio_indicador = 2016
dfDesempleados2016 = iteracion_callback(anio_indicador, 'INDICADOR', 49, 1, 'NRO_PERSONAS')
dfDesempleados2016.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfDesempleados2018 = iteracion_callback(anio_indicador, 'INDICADOR', 65, 1, 'NRO_PERSONAS')
dfDesempleados2018.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfDesempleados2020 = iteracion_callback(anio_indicador, 'INDICADOR', 65, 1, 'NRO_PERSONAS')
dfDesempleados2020.to_excel("../../data_preprocesada_N1/desempleados_población_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfDesempleados2016, dfDesempleados2018, dfDesempleados2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/desempleados_barrios_madrid.xlsx")

## Precio de la Vivienda (Nuevas, Usadas) 2016, 2018, 2020

Continuando con la integración de los datos, en la siguiente URL se extraerán los valores del indicador **Precio de la Vivienda**

> http://www-2.munimadrid.es/CSE6/control/seleccionDatos?numSerie=5040201060

In [130]:
#Función para lectura iterativa de excel Compra-Venta Vivienda
def iteracion_callback_f2(df, year, indicadorName, unidad):
    dfPreProcesadoFull = pd.DataFrame()
    i = 0
    contador_x = 0
    contador_y = 172
    while i < 1:
        # Inicializa dataframes
        dfAux = pd.DataFrame()
        # Inicializa iteradores de filas
        contadorX = contador_x + 1
        contadorY = contadorX + contador_y        
        dfAux = df[contadorX:contadorY]
        dfAux.reset_index(inplace=True, drop=True)
        # Pasar columnas como valores del dataframe bajo el campo Barrios
        df2 = pd.melt(dfAux, id_vars=[indicadorName], 
                      var_name="TIPO_VIVIENDA", value_name=unidad)        
        # Agrega columna anio
        df2['ANIO'] = year
        # Agrega columna Indicador
        df2['INDICADOR'] = 'PRECIO COMPRA VIVIENDA'        
        auxBarrio = df2['BARRIOS'].str.split('\. ', 1, expand=True)
        df2['BARRIOS'] = auxBarrio[1]
        # Strings a mayúsculas
        df2 = df2.apply(lambda x: x.str.upper() if x.dtype == "object" else x)  
        #Contadores
        contador_x = contadorY        
        i = i + 1
    return df2

In [131]:
#Lectura de excel precios vivienda 2016.xls
df2016 = pd.read_excel('../../original_data/precio_viviendas_barrios_2016.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

#Lectura de excel precios vivienda 2018.xls
df2018 = pd.read_excel('../../original_data/precio_viviendas_barrios_2018.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

#Lectura de excel precios vivienda 2020.xls
df2020 = pd.read_excel('../../original_data/precio_viviendas_barrios_2020.xls', 
                   sheet_name=0, 
                   skiprows= 7 ,
                   usecols="A,B,C")

# Dataset Nivel 1
anio_indicador = 2016
dfPrecioVivienda2016 = iteracion_callback_f2(df2016, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2016.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfPrecioVivienda2018 = iteracion_callback_f2(df2018, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2018.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2020
dfPrecioVivienda2020 = iteracion_callback_f2(df2020, anio_indicador, 'BARRIOS', 'PRECIO(EUROS/M2)')
dfPrecioVivienda2020.to_excel("../../data_preprocesada_N1/precio_vivienda_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfPrecioVivienda2016, dfPrecioVivienda2018, dfPrecioVivienda2020]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/precio_vivienda_barrios_madrid.xlsx")


## Turismo (Hospedaje) (Nuevas, Usadas) 2016, 2018, 2020

Con la siguiente URL se extraerán los valores correspondientes al indicador **Turismo - Hospedaje**

> https://www.kaggle.com/code/rusiano/housing-crisis-and-tourism-in-madrid/data?select=listings.csv

In [132]:
# Lectura del CSV de Airbnb
dfHospedaje = pd.read_csv('../../original_data/airbnb_turismo_barrios.csv')
dfHospedaje['ANIO'] = pd.to_datetime(dfHospedaje['last_review']).dt.year
dfHospedaje = dfHospedaje[['id','neighbourhood','latitude', 'longitude', 'room_type', 'ANIO']]

dfHospedaje = dfHospedaje.rename(columns = {'id': 'ID',
                                  'neighbourhood': 'BARRIO',
                                  'latitude': 'LATITUD',
                                  'longitude': 'LONGITUD',
                                  'room_type': 'CATEGORIA'}, inplace = False)
# Todo mayuscula
dfHospedaje['BARRIO'] = dfHospedaje['BARRIO'].str.upper()
dfHospedaje['CATEGORIA'] = dfHospedaje['CATEGORIA'].str.upper()

# Elimina valores NA en el campo fecha
dfHospedaje = dfHospedaje.dropna()

# Filtra anios desde el 2018
dfHospedaje = dfHospedaje[dfHospedaje['ANIO'].isin([2016, 2018, 2019])]

# Verifica datos duplicados
dfHospedajeDeleteDuplicados = dfHospedaje[dfHospedaje.duplicated(['LATITUD', 'LONGITUD'])]

# Elimina datos duplicados
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 17453051 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 17809905 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 20011461 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 22312525 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 23576224 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 35217403 ].index, inplace = True)
dfHospedaje.drop(dfHospedaje[dfHospedaje['ID'] == 38310467 ].index, inplace = True)


In [133]:
# Dataset Nivel 1
anio_indicador = 2016
dfHospedaje2016 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
dfHospedaje2016.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2018
dfHospedaje2018 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
dfHospedaje2018.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

anio_indicador = 2019
dfHospedaje2019 = dfHospedaje[dfHospedaje['ANIO'].isin([anio_indicador])]
dfHospedaje2019.to_excel("../../data_preprocesada_N1/hospedaje_" + str(anio_indicador) + ".xlsx")

# Dataset Nivel 2
finalDataset = pd.DataFrame()
lstFrames = [dfHospedaje2016, dfHospedaje2018, dfHospedaje2019]
finalDataset = pd.concat(lstFrames)
finalDataset.to_excel("../../data_preprocesada_N2/hospedaje_barrios_madrid.xlsx")
